# Webscraping Immowelt

## Projektbeschreibung

Ziel dieses Projektes ist es zu Verkauf stehende Häuser von Immowelt zu scrapen, die gesammelten Daten auszuwerten und eine Regression zur bestmöglichen Vorhersage der Hauspreise durchzuführen. Außerdem werden die Immobiliendaten mit dem Bauzins angereichert, so dass Zusammenhänge zwischen Zins und Preis untersucht werden. 

In diesem ersten Notebook wird das Webscraping der zu Verkauf stehenden Häuser in Immowelt betrachtet und durchgeführt. Für das Scraping werden sieben deutsche Städte und ein Umkreis von 50 km um sie herum betrachtet: Berlin, Hamburg, München, Köln, Frankfurt am Main, Stuttgart und Leipzig.

Die sinnvolle Reihenfolge, in welcher die Notebooks zu betrachten sind:
1. Webscraping Immowelt
2. Daily Immowelt
3. Webscraping Comdirect
4. Daily Comdirect
5. Regression (zzgl. weiterer Analysen)

## Inhaltsverzeichnis

1. [Installationen](#1-installationen)
2. [URL Untersuchung](#2-url-untersuchung)
3. [Webscraping](#3-webscraping)
    1. [Suchergebnisliste für eine URL](#31-suchergebnisliste-für-eine-url)
    2. [Suchergebnisliste für alle URL-Kombinationen](#32-suchergebnisliste-für-alle-url-kombinationen)
    3. [Exposé für eine URL](#33-exposé-für-eine-url)
    4. [Update For Schleife Suchergebnisliste mit Exposé für alle URL-Kombinationen](#34-update-for-schleife-suchergebnisliste-mit-exposé-für-alle-url-kombinationen)

## 1. Installationen

In [2]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import lxml
from lxml import html
from lxml import etree

import re
import csv
from csv import writer
from scipy import stats


## 2. URL Untersuchung

Im ersten Schritt wurde eine beispielhafte Immobiliensuche für Häuser zum Kauf in Berlin in die Suchmaske eingegeben. 

URL (1): https://www.immowelt.de/liste/berlin/haeuser/kaufen?sort=relevanz

In der Ergebnisliste waren zunächst die Neubauprojekt auffällig, die u.a. keine eindeutige Angabe zum Preis hatten. Da dies für die spätere Analyse wichtig ist, wurde entschieden, die Neubauprojekt wegzufiltern. Bei der Filteroption wurden in dem Zuge ebenfalls Immobilien rausgefiltert, die zwangsversteigert werden. Dabei hat sich die URL verändert.

URL (2): https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&sd=DESC&sf=RELEVANCE&sp=1

Um aus der Suche möglichst viele Immoblien abzudecken, wird der Suchradius auf 50 Kilometer eingestellt. Auch hierbei hat sich die URL wieder verändert.

URL (3): https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=1

Anhand der finalen URL ist gut zu erkennen, dass u.a. der Ort, die Objektart, der Suchradius sowie die Seitenzahl eingebettet sind. Das sind gute Vorraussetzungen, um im nächsten Schritt, die weiteren Suchkriterien zu definieren.




### Definierung der Suchkriterien

Die URL wird zunächst in ihre Bestandteile aufgeteilt.

Für die variablen Teile werden folgende Umfänge beschrieben:

| Variable | Beschreibung |  
| :--- | :--- |  
| Ort | Für den Ort werden sieben Großstädte in Deutschland gewählt. |  
| Umkreis | Der Suchradius wird bei 50 Kilometer belassen. |  
| Objekt | Bei der Objektart wird das Projekt auf Häuser eingeschränkt. Theoretisch können hier auch Wohnungen ["wohnungen"] eingetragen werden. |  
| Seite  | Hier war die erste Idee den letzten Seitenbutton auszulesen, um darüber die Seitenanzahl für jede Suche festzulegen. Im späteren Verlauf wird festgestellt, dass der Button nicht ausgelesen werden konnte. Daher wird zunächst, auch in Anbetracht der Durchlaufzeit, manuell die Seitenzahl auf 50 definiert, mit dem Wissen, dass es durchaus mehr Seiten gäbe. |

In [3]:
Ort = ["berlin", "frankfurt-am-main", "hamburg", "koeln", "leipzig", "muenchen", "stuttgart"]
Umkreis = 50
Objekt = ["haeuser"]
Seite = list(range(50))

Die verbleibenden Bestandteile der URL werden so zu Variablen eingeteilt, sodass im weiteren Verlauf die URL wieder komplett zusammengesetzt werden kann.

In [4]:
Website = "https://www.immowelt.de/liste/"
Slash = "/"
Snippet1 = "/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r="
Snippet2 = "&sd=DESC&sf=RELEVANCE&sp="

Als Beispiel wird die URL für Ort[0] = Berlin, Objekt[0] = haeuser und die erste Seite (Seite[0]+1) generiert. Das +1 bei der Seite ist notwendig, da eine Liste immer bei 0 anfängt, jedoch die Seitenzahl bei Immowelt.de bei 1 beginnt.


In [5]:
url = Website + Ort[0] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[0]+1)

print(url)

https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=1



## 3. Webscraping

### 3.1. Suchergebnisliste (für eine URL)

Die im vorherigen Schritt generierte URL wird die erste Grundlage für das Webscraping sein.

![Immowelt_Ergebnisliste_Berlin.png](C:/Users/haimi/Documents/GitHub/WebSocialMediaAnalytics/screenshots/Immowelt_Ergebnisliste_Berlin.png)

Es ist schnell zu erkennen, dass alle Objekte folgende Attribute besitzen (von oben nach unten).

1. Objekttitel
2. Entfernung zum Suchort
3. Grundstücksfläche
4. Objektbeschreibung
5. Preis
6. Wohnfläche
7. Zimmeranzahl

#### Untersuchung HTML


Über das Entwicklertool wird eine erste visuelle Sichtung des HTML-Textes durchgeführt.

Hierbei fällt auf, dass jedes Objekt einer ID zugeordnet ist. Die angezeigten Attribute sind übermäßig in den div-Klassen "KeyFacts-efbce" und "estateFacts-f11b0" eingebettet. Diese werden später für das Webscraping der einzelnen Attribute relevant sein.

Um ein Gefühl für die Anzahl der Datensätze zu bekommen, wird vorweg die Anzahl der Objekt-IDs für die URL geprüft. In diesem Fall wird zunächst die HTML der URL ausgegeben. Mit XPath wird ein "Startpunkt" im HTML definiert, um von da aus gemäß dem vorgegebenen Pfad den erwünschten Teil zu erreichen, hier nämlich die ID.

In [9]:
# URL Aufrufen
page = requests.get(url)
 
# Parsen der Seite URL
tree = html.fromstring(page.content)
 
# Alle IDs der Seite URL in der Liste id speichern. IDs sind immer eindeutig und nur einmal vergeben

id = tree.xpath(".//a/@id")

# Printen der Liste um es zu überprüfen
for e in range(1):
    print(id)

print("Anzahl IDs auf einer Seite: "+str(len(id)))

print ("Anzahl IDs gesamt ca. (bei "+str(len(Seite))+" Seiten pro Ort): "+str((len(id)*len(Ort)*len(Seite))))

['28zn659', '28jr359', '274he5l', '284bu5a', '28u535d', '28y9q52', '279kt5y', '27a225p', '28g8n58', '27w5v5j', '28lxe59', '28z335d', '28mq35d', '28crz5d', '27nhe5l', '28xyz53', '26pew5r', '28kzy5b', '28z9y5c', '27yc95r']
Anzahl IDs auf einer Seite: 20
Anzahl IDs gesamt ca. (bei 50 Seiten pro Ort): 7000


#### Webscraping ausgewählter Attribute

Nach dem gleichen Prinzip wie bei der Objekt-ID werden folgende Attribute ausgelesen:


| Attribut | Variable |  
| :--- | :--- |  
| Entfernung zum Suchort | MaxOrt |  
| Preis | prices |  
| Wohnfläche | area |  
| Zimmeranzahl  | rooms |


Am Beispiel von "MaxOrt" (Entfernung zum Suchort) wird für XPath der Startpunkt div[@class="estateFacts-f11b0"] festgelegt. Danach führt der Pfad über die Unterstruktur zum div [1]. Die Angabe der Ziffer ist hierbei wichtig, da es mehrere divs unter dem Startpunkt gibt. Weiter führt der Pfad zum span, wo dann zuletzt mit /text die Text-Inhalte ausgelesen werden. Für die anderen Attribute kann direkt auf div[@data-test=*] zugegriffen werden.


In [11]:
MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')
prices = tree.xpath('//div[@data-test="price"]/text()')
area = tree.xpath('//div[@data-test="area"]/text()')
rooms = tree.xpath('//div[@data-test="rooms"]/text()')

# Printen aller Ergebnisse die man aus dieser 1. Seite rausgelesen hat
for e in range(1):
    print(MaxOrt)
for e in range(1):
    print(prices)
for e in range(1):
    print(area)
for e in range(1):
    print(rooms)


['max. 15 km | Berlin (Zehlendorf)', 'max. 30 km | Edelweißstr. 5, Woltersdorf', 'max. 10 km | Berlin / Lankwitz (Steglitz)', 'max. 10 km | Berlin (Niederschönhausen)', 'max. 10 km | Berlin (Buckow)', 'max. 10 km | Berlin (Lankwitz)', 'max. 10 km | Berlin (Lichterfelde)', 'max. 10 km | Berlin / Schmargendorf (Schmargendorf)', 'max. 10 km | Berlin (Westend)', 'max. 15 km | Berlin / Biesdorf (Biesdorf)', 'max. 15 km | Berlin (Lichterfelde)', 'max. 15 km | Berlin (Rudow)', 'max. 15 km | Rhodeländer Weg 26 B, Berlin (Rudow)', 'max. 15 km | Berlin (Wittenau)', 'max. 15 km | Berlin (Lichterfelde)', 'max. 15 km | Berlin (Kaulsdorf)', 'max. 15 km | Berlin (Köpenick)', 'max. 15 km | Berlin (Karow)', 'max. 15 km | Berlin (Lichterfelde)', 'max. 15 km | Puchanstraße 33, Berlin (Köpenick)']
['7.790.000 €', '450.000 €', '690.000 €', '835.500 €', '548.000 €', '490.000 €', '1.700.000 €', '7.500.000 €', '1.750.000 €', '795.000 €', '790.000 €', '950.000 €', '444.444 €', '750.000 €', '1.140.000 €', '549.

Bei allen Attributen sind Datenaufbereitungsschritte für weitere Analysen notwendig. Diese werden im Rahmen der Regression durchführt. Einzig bei MaxOrt wurde entschieden, den Ort hinter der Kilometeranzahl, bereits jetzt zu entfernen. 

Beispiel für MaxOrt: 'max. 15 km | Berlin (Zehlendorf)'

In [12]:
#dafür geht man von der ersten Ziffer bis zum | und löscht den Rest raus mit dem Befehl
#MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")-1] die -1 steht dafür dass man noch eine 
#Ziffer mehr löscht von rechts nach links Gezeählt ab dem Zeichen "|"

#da man dies für jeden Wert in der Liste MaxOrt machen muss hier wieder eine For schleife mit range als Länge 
# der Liste
MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')

for e in range(len(MaxOrt)):
    MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")-1]
    
# für die Überprüfung wieder printen
for e in range(1):
    print(MaxOrt)


['max. 15 km', 'max. 30 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 10 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km', 'max. 15 km']


Zu einem früheren Zeitpunkt des Projektes (13.12.22) waren die Listen für die einzelnen Attribute nicht gleich lang. Für ein Objekt (ID 28auw54) gab es keine Angabe zu der Zimmeranzahl. Bei XPath zieht der Befehl jeden Wert, wo etwas drinsteht raus. Wenn nichts drinsteht, überspringt er es. Das heißt wenn nichts drinsteht, wird der Wert "Nothing" nicht in der Liste gespeichert. Somit können die Listen unterschiedliche Längen haben. Da dieser Fall auch öfters oder für andere Attribute vorkommen kann, darf die fehlende Angabe nicht übersprungen werden, sondern muss durch ein "n.a." gefüllt werden.

In [14]:
#Anzahl der Zeilen in der Liste Printen um zu überprüfen ob alle die gleiche Länge haben

print ("Anzahl der Listenelemente ID:", len(id))
print ("Anzahl der Listenelemente Preise:", len(prices))
print ("Anzahl der Listenelemente Fläche:", len(area))
print ("Anzahl der Listenelemente Zimmer:", len(rooms))

Anzahl der Listenelemente ID: 20
Anzahl der Listenelemente Preise: 20
Anzahl der Listenelemente Fläche: 20
Anzahl der Listenelemente Zimmer: 20


Um das ganze Skript zu automatisieren, müssen die Listen alle die gleiche Länge haben, da später mit einer Matrix gearbeitet wird. Der Lösungsansatz beinhaltet die IDs, da diese pro Objekt eindeutig zugeordnet sind. Daher werden im XPath für **rooms** und **area** als Startpunkt die ID festgelegt. Von da aus wird der Pfad bis zum erwünschten Attribut festgelegt.

In diesem Beispiel hat die ID 28auw54 am 13.12.2022 keine Zimmeranzahl. Mit dem folgenden Befehl wird auch der fehlende Wert als "leerer" Wert ausgelesen.

In [15]:
rooms= tree.xpath('//a[@id= "28auw54"]/div[2]/div[1]/div[1]/div[3]/text()')

print(rooms)

[]


Der oben definierte Befehl muss für **rooms** und **area** so angepasst werden, sodass die ID als Variable eingesetzt werden kann. Dabei steht anstelle der ID ein "%s" und die Variable steht am Ende vom Befehl als "%id[e]":

In [17]:
# rooms[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[3]/text()'% id[e])

Dadurch wird jetzt nach jeder ID auf der Seite gesucht und über den Pfad in den Unterstrukturen die Zimmeranzahl oder die Fläche ausgelesen.

Um eine dynamische Allokierung der Listen **rooms** und **area** zu erhalten werden Matrixen erstellt mit der Spaltenanzahl 2 (range(1)) und der Zeilenanzahl analog der Anzahl der IDs (len(id))

Man bezieht sich dann unten nur auf eine Liste indem man z.B. rooms[e][0] macht. Die 0 steht für Spalte 1
Wenn man sich nur in einer Spalte hier Spalte (1) bewegt, erhält man sozusagen eine Liste.

In [18]:
rooms = [[0 for s in range(1)] for r in range(len(id))]
area= [[0 for s in range(1)] for r in range(len(id))]


id = tree.xpath(".//a/@id")
prices = tree.xpath('//div[@data-test="price"]/text()')
MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')


# hier die Schleife mit dem Variablenbefehl
for e in range(len(id)):
    rooms[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[3]/text()'% id[e])
    area[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[2]/text()'% id[e])


for e in range(len(MaxOrt)):
    MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")]


# Printen zum Überprüfen
# am 13.12.2022 sind trotz fehlender Informationen in den Zimmern bei den Objekten (Häuser) jeder Listenplatz ausgefüllt (auch mit Nothing Werten)

for e in range(1):     
    print(id)
print ("Anzahl der Listenelemente ID:", len(id))

for e in range(1):     
    print(MaxOrt)
print ("Anzahl der Listenelemente Max Ortsabweichung:", len(MaxOrt))
for e in range(1):     
    print(prices)
print ("Anzahl der Listenelemente Preise:", len(prices))
for e in range(1):     
    print(area)
print ("Anzahl der Listenelemente Fläche:", len(area))
for e in range(1):     
    print(rooms)
print ("Anzahl der Listenelemente Zimmer:", len(rooms))

['28zn659', '28jr359', '274he5l', '284bu5a', '28u535d', '28y9q52', '279kt5y', '27a225p', '28g8n58', '27w5v5j', '28lxe59', '28z335d', '28mq35d', '28crz5d', '27nhe5l', '28xyz53', '26pew5r', '28kzy5b', '28z9y5c', '27yc95r']
Anzahl der Listenelemente ID: 20
['max. 15 km ', 'max. 30 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 10 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ', 'max. 15 km ']
Anzahl der Listenelemente Max Ortsabweichung: 20
['7.790.000 €', '450.000 €', '690.000 €', '835.500 €', '548.000 €', '490.000 €', '1.700.000 €', '7.500.000 €', '1.750.000 €', '795.000 €', '790.000 €', '950.000 €', '444.444 €', '750.000 €', '1.140.000 €', '549.000 €', '1.590.000 €', '727.000 €', '995.000 €', '344.000 €']
Anzahl der Listenelemente Preise: 20
[[['780 m²']], [['111 m²']], [['323 m²']], [['90 m²']], [['90 m²']], [['80 m²']], [['1

Als Ergebnis erhält man für jedes Attribut eine Liste, die genauso viele Werte hat wie IDs vorhanden sind. Damit ist der Webscraping Prozess für die Suchergebnisliste für die Bsp-URL abgeschlossen.

### 3.2. Suchergebnisliste (für alle URL-Kombinationen)

In diesem Kapitel wird das Web Scraping für alle URL-Kombinationen mit den definierten Suchkriterien angegangen. Dafür wird die variabilisierte URL aus Kapitel 2 benötigt.

**For Schleife URL**

Zunächst wird wieder eine Matrix allokiert, mit der Spaltenanzahl len(Ort) und der Zeilenanzahl len(Seite). Für jeden Ort erhält man dann die URL für jede Seite in den Zeilen.

In [21]:
a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
for i in range(len(Ort)):
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        
#Printen zum Überprüfen 

for i in range(len(Ort)):
    for j in range(len(Seite)):
        print(a[i][j])


https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=1
https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=2
https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=3
https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=4
https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=5
https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=6
https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r=50&sd=DESC&sf=RELEVANCE&sp=7
https://www.immowelt.de/liste/berlin/haeuser/kaufen?d=true&efs=NEW_BU

An dieser Stelle war die Überlegung die Range der **Seite** über den letzten Seitenzahl-Button zu definieren. Dies ist nicht gelungen.

In [12]:
#Versuche zum Auslesen letzter Seitenzahl-Button

button_v1 = tree.xpath('//div[@class="Pagination-190de"]/div[1]/button[5]/span/text()')

print("Button Versuch 1: "+str(button_v1))

button_v2 = tree.xpath('//button[@class="Button-c3851 secondary-47144 navNumberButton-d264f"]/span/text()')

print("Button Versuch 2: "+str(button_v2))



Button Versuch 1: []
Button Versuch 2: []


Im nächsten Schritt werden die Befehle für das Auslesen der einzelnen Attribute mit der "URL For Schleife" kombiniert. Am Ende werden die einzelnen Daten in ein Datenframe gebracht und als CSV-Datei abgespeichert.

In [134]:
# Ermittlung aller IDs für die Gesamtanzahl der Zeilen in der Matrix

AnzahlElementebyID = 0

a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
for i in range(len(Ort)):
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        page = requests.get(a[i][j])
        tree = html.fromstring(page.content)
        id = tree.xpath(".//a/@id")
        
        AnzahlElementebyID = AnzahlElementebyID+len(id)



#Die Matrix b hat als Länge der Zeilen die Gesamtanzahl der Objekte über alle Seiten (AnzahlElementebyID)

#Auslesen der einzelnen Attribute für jede URL-Kombination

b = [[0 for s in range(15)] for r in range(len(Ort)*AnzahlElementebyID)]
a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
laenge = 0
for i in range(len(Ort)):    
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        page = requests.get(a[i][j])
        tree = html.fromstring(page.content)
        id = tree.xpath(".//a/@id")
        prices = tree.xpath('//div[@data-test="price"]/text()')
        MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')
        for e in range(len(MaxOrt)):
            MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")-1] 
        
        rooms = [[0 for s in range(1)] for r in range(len(id))]
        area= [[0 for s in range(1)] for r in range(len(id))]
        
        for e in range(len(id)):
            rooms[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[3]/text()'% id[e])
            area[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[2]/text()'% id[e])
            
            
        for r in range(len(id)):
             b[laenge+r][0]= id[r] 
             b[laenge+r][1]= MaxOrt[r]     
             b[laenge+r][2]= prices[r]      
             b[laenge+r][3]= Ort[i]
             b[laenge+r][4]=Umkreis
             b[laenge+r][5]= area[r][0]
             b[laenge+r][6]= rooms[r][0]
             
             
           
             
        # laenge ist die aktuelle Anzahl an Häusern in der For Schleife 
        # for r in range(len(id)): für die erste Seite fängt die Schleife bei b[0+r][Beliebiger Platzhalter] an, 
        # die zweite Seite ist die Häuseranzahl der ersten Seite laenge=0+len(id) also z.B laenge=20
        # dann fängt für die zweite Seite die b Matrix bei b[20+r][beliebiger Platzhalter] an und geht für
        # r = len(id) also die Anzahl der Häuser ab und addiert diese auf den letzten Wert der vorherigen Seite 
        # siehe  b[laenge+r][Beliebiger Platzhalter]  
        # laenge = laenge + len(id) 
        
              
# Erstellung des Dataframes und speichern als CSV-Datei    

data = {
    'ID': [],
    'Ort': [],
    'Umkreis': [],
    'MaxOrt': [],
    'Preis': [],
    'Fläche': [],
    'Zimmer': [],
}

for q in range(AnzahlElementebyID):
    data['ID'].append(b[q][0])
    data['Ort'].append(b[q][3])
    data['Umkreis'].append(b[q][4])
    data['MaxOrt'].append(b[q][1])
    data['Preis'].append(b[q][2])
    data['Fläche'].append(b[q][5])
    data['Zimmer'].append(b[q][6])
    
    
df=pd.DataFrame(data, columns=['ID','Ort','Umkreis','MaxOrt','Preis','Fläche','Zimmer']) 
df.to_csv('Immobilien.csv')

### 3.3. Exposé (für eine URL)

In den einzelnen Exposés der Objekte sind weitere Attribute einzusehen. Nach dem gleichen Prinzip wie bei der Suchergebnisliste wird zuerst die URL untersucht und die zu auslesenden Attribute aufgelistet.


1. Grundstücksfläche
2. Kategorie
3. Geschosse
4. Baujahr
5. Effizienzklasse
6. Energieträger
7. Heizungsart

Die URL setzt sich aus der Website + expose + ID zusammen: https://www.immowelt.de/expose/28auw54

Wieder liegen gute Voraussetzungen vor, die URL zu variablisieren.

In [ ]:
#Beispiel URL für die erste ID

website = "https://www.immowelt.de/expose/"
haus_url = website + df["ID"].iloc[0]

print(haus_url)


Analog des Auslesens der Attribute aus der Sucherergebnisse wird auch für für das Webscraping der Exposé-Seiten zunächst XPath verwendet.

| Attribut | Variable |  
| :--- | :--- |  
| Grundstücksfläche | Grundstücksfläche |  
| Kategorie | Kategorie |  
| Geschosse | Etagen |  
| Baujahr  | Baujahr |
| Effizienzklasse  | Effizienzklasse |
| Energieträger  | Energieträger |
| Heizungsart  | Heizungsart |

In [136]:
expose = requests.get(haus_url)
ex_tree = html.fromstring(expose.content)

Grundstücksfläche = ex_tree.xpath('//div[@class="flex ng-star-inserted"]/div[3]/span/text()')
Kategorie = ex_tree.xpath('//div[@class="equipment card-content ng-star-inserted"]/sd-cell[1]/sd-cell-row/sd-cell-col/p[2]/text()')
Etagen = ex_tree.xpath('//div[@class="equipment card-content ng-star-inserted"]/sd-cell[2]/sd-cell-row/sd-cell-col/p[2]/text()')
Baujahr = ex_tree.xpath('//div[@class="equipment card-content ng-star-inserted"]/sd-cell[3]/sd-cell-row/sd-cell-col/p[2]/text()')
Effizienzklasse = ex_tree.xpath('//div[@class="energy_information ng-star-inserted"]/sd-cell[5]/sd-cell-row/sd-cell-col/p[2]/text()')
Energieträger = ex_tree.xpath('//div[@class="card__cell pb-100 pb-75:400"]/sd-cell[1]/sd-cell-row/sd-cell-col/p[2]/text()')
Heizungsart = ex_tree.xpath('//div[@class="card__cell pb-100 pb-75:400"]/sd-cell[2]/sd-cell-row/sd-cell-col/p[2]/text()')

print("Grundstücksfläche: " + str(Grundstücksfläche))
print("Kategorie: " + str(Kategorie))
print("Etagen: " + str(Etagen))
print("Baujahr: " + str(Baujahr))
print("Effizienzklasse: " + str(Effizienzklasse))
print("Energieträger: " + str(Energieträger))
print("Heizungsart: " + str(Heizungsart))


Im weiteren Verlauf des Projektes gab es jedoch die Erkenntnis, dass nicht jedes Attribut immer an der gleichen Stelle im HTML steht, sodass z.B. auch mal das Baujahr dort steht, wo bei einem anderen Objekt die Etagenanzahl vermerkt ist. Um dieses Problem zu umgehen wird gezielt nach den Textbausteinen der Attribute im HTML gesucht.

In [ ]:
Grundstücksfläche= ex_tree.xpath(".//p[text()='Grundstücksfläche']")
Kategorie= ex_tree.xpath(".//p[text()='Kategorie']")
Etagen= ex_tree.xpath(".//p[text()='Geschosse']")
Baujahr= ex_tree.xpath(".//p[text()='Baujahr']")
Effizienzklasse= ex_tree.xpath(".//p[text()='Effizienzklasse']")
Energieträger= ex_tree.xpath(".//p[text()='Energieträger']")
Heizungsart= ex_tree.xpath(".//p[text()='Heizungsart']")
    
     
    b[i][7]= ex_tree.xpath('//div[@class="flex ng-star-inserted"]/div[3]/span/text()') 
        
    
    if (len(Kategorie)) >= 1:  
        b[i][8] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Kategorie"])]/p[2]/text()')
    else: 
        b[i][8]= "n.a" 
    
    if (len(Etagen)) >= 1:  
        b[i][9] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Geschosse"])]/p[2]/text()')
    else:
        b[i][9]= "n.a" 

    if (len(Baujahr)) >= 1:  
        b[i][10]= ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Baujahr"])]/p[2]/text()')
    else:
        b[i][10]= "n.a" 
   
    if (len(Effizienzklasse)) >= 1:  
        b[i][11] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Effizienzklasse"])]/p[2]/text()')
    else: 
        b[i][11]= "n.a" 
   
    if (len(Energieträger)) >= 1:  
        b[i][12] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Energieträger"])]/p[2]/text()')
    else:
        b[i][12]= "n.a"
    
    if (len(Heizungsart)) >= 1:  
        b[i][13] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Heizungsart"])]/p[2]/text()')
    else:
      b[i][13]= "n.a"  

### 3.4. Update For Schleife Suchergebnisliste mit Exposé (für alle URL-Kombinationen)

Die For Schleife für die Sucherergebnisliste wird nun mit den Befehlen aus der Exposé Untersuchung ergänzt. Am Ende werden die Daten wieder in ein Dataframe gebracht und als CSV abgespeichert.

In [138]:
# Ermittlung aller IDs für die Gesamtanzahl der Zeilen in der Matrix

AnzahlElementebyID=0

a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
for i in range(len(Ort)):
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        page = requests.get(a[i][j])
        tree = html.fromstring(page.content)
        id = tree.xpath(".//a/@id")
        
        AnzahlElementebyID=AnzahlElementebyID+len(id)


#Die Matrix b hat als Länge der Zeilen die Gesamtanzahl der Objekte über alle Seiten (AnzahlElementebyID)

#Auslesen der einzelnen Attribute für jede URL-Kombination

b = [[0 for s in range(15)] for r in range(len(Ort)*AnzahlElementebyID)]
a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
laenge = 0
for i in range(len(Ort)):    
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        page = requests.get(a[i][j])
        tree = html.fromstring(page.content)
        id = tree.xpath(".//a/@id")
        prices = tree.xpath('//div[@data-test="price"]/text()')
        MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')
        for e in range(len(MaxOrt)):
            MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")-1] 
        
        rooms = [[0 for s in range(1)] for r in range(len(id))]
        area= [[0 for s in range(1)] for r in range(len(id))]
        
        for e in range(len(id)):
            rooms[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[3]/text()'% id[e])
            area[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[2]/text()'% id[e])
            
        
        
        
        for r in range(len(id)):
             b[laenge+r][0]= id[r] 
             b[laenge+r][1]= MaxOrt[r]     
             b[laenge+r][2]= prices[r]      
             b[laenge+r][3]= Ort[i]
             b[laenge+r][4]=Umkreis
             b[laenge+r][5]= area[r][0]
             b[laenge+r][6]= rooms[r][0]
             
             
           
             
        #laenge ist die Aktuelle Anzahl Häusern in der For Schleife 
        # for r in range(len(id)): für die erste Seite fängt es bei b[0+r][Beliebiger Platzhalter] an dann 
        # die zweite Seite ist die Häuseranzahl der ersten Seite laenge=0+len(id) also z.B laenge=20
        # dann fängt für die zweite Seite die b Matrix bei b[20+r][beliebiger Platzhalter] an und geht für
        # r = len(id) also die Anzahl der Häuser ab und addiert diese auf den letzten Wert der vorherigen Seite 
        # siehe  b[laenge+r][Beliebiger Platzhalter]  
        # laenge = laenge + len(id) 
        
              
#Die Daten von der Exposéseite des Objektes scrapen

haus_url= [[0 for j in range(100000)] for i in range(AnzahlElementebyID+100)]
website = "https://www.immowelt.de/expose/"

for i in range(AnzahlElementebyID):
    haus_url[0][i] = website + str(b[i][0])
    
    
for i in range(AnzahlElementebyID):    
    expose = requests.get(haus_url[0][i])
    ex_tree = html.fromstring(expose.content) 
    
    Grundstücksfläche= ex_tree.xpath(".//p[text()='Grundstücksfläche']")
    Kategorie= ex_tree.xpath(".//p[text()='Kategorie']")
    Etagen= ex_tree.xpath(".//p[text()='Geschosse']")
    Baujahr= ex_tree.xpath(".//p[text()='Baujahr']")
    Effizienzklasse= ex_tree.xpath(".//p[text()='Effizienzklasse']")
    Energieträger= ex_tree.xpath(".//p[text()='Energieträger']")
    Heizungsart= ex_tree.xpath(".//p[text()='Heizungsart']")
    
     
    b[i][7]= ex_tree.xpath('//div[@class="flex ng-star-inserted"]/div[3]/span/text()') 
        
    
    if (len(Kategorie)) >= 1:  
        b[i][8] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Kategorie"])]/p[2]/text()')
    else: 
        b[i][8]= "n.a" 
    
    if (len(Etagen)) >= 1:  
        b[i][9] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Geschosse"])]/p[2]/text()')
    else:
        b[i][9]= "n.a" 

    if (len(Baujahr)) >= 1:  
        b[i][10]= ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Baujahr"])]/p[2]/text()')
    else:
        b[i][10]= "n.a" 
   
    if (len(Effizienzklasse)) >= 1:  
        b[i][11] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Effizienzklasse"])]/p[2]/text()')
    else: 
        b[i][11]= "n.a" 
   
    if (len(Energieträger)) >= 1:  
        b[i][12] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Energieträger"])]/p[2]/text()')
    else:
        b[i][12]= "n.a"
    
    if (len(Heizungsart)) >= 1:  
        b[i][13] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Heizungsart"])]/p[2]/text()')
    else:
      b[i][13]= "n.a"  
   
# Erstellung des Dataframes und speichern als CSV-Datei    

data = {
    'ID': [],
    'Ort': [],
    'Umkreis': [],
    'MaxOrt': [],
    'Preis': [],
    'Fläche': [],
    'Zimmer': [],
    'Grundstücksfläche': [],
    'Kategorie': [],
    'Etagen': [],
    'Baujahr': [],
    'Effizienzklasse': [],
    'Energieträger': [],
    'Heizungsart': [],
}

for q in range(AnzahlElementebyID):
    data['ID'].append(b[q][0])
    data['Ort'].append(b[q][3])
    data['Umkreis'].append(b[q][4])
    data['MaxOrt'].append(b[q][1])
    data['Preis'].append(b[q][2])
    data['Fläche'].append(b[q][5])
    data['Zimmer'].append(b[q][6])
    data['Grundstücksfläche'].append(b[q][7])
    data['Kategorie'].append(b[q][8])
    data['Etagen'].append(b[q][9])
    data['Baujahr'].append(b[q][10])
    data['Effizienzklasse'].append(b[q][11])
    data['Energieträger'].append(b[q][12]) 
    data['Heizungsart'].append(b[q][13]) 
      
     
     
     
     
df=pd.DataFrame(data, columns=['ID','Ort','Umkreis','MaxOrt','Preis','Fläche','Zimmer','Grundstücksfläche','Kategorie','Etagen','Baujahr','Effizienzklasse','Energieträger','Heizungsart']) 
df.to_csv('Immobilien.csv')

Die CSV wird Datengrundlage für die Regression sein. Für weitere Analysen wie beispielsweise bezüglich der Inventardauer wird der Code für eine tägliche Abfrage verwendet.

In [139]:
df

,ID,Ort,Umkreis,MaxOrt,Preis,Fläche,Zimmer,Grundstücksfläche,Kategorie,Etagen,Baujahr,Effizienzklasse,Energieträger,Heizungsart
0,28zn659,berlin,50,max. 15 km,7.790.000 €,[780 m²],[30 Zi.],[1.696 m²],[Villa],[5 Geschosse],[1890],n.a,n.a,n.a
1,28jr359,berlin,50,max. 30 km,450.000 €,[111 m²],[5 Zi.],[1.100 m²],[Einfamilienhaus],[2 Geschosse],[1936],[H],"[Gas, Kohle]",[Zentralheizung]
2,274he5l,berlin,50,max. 10 km,690.000 €,[323 m²],[8 Zi.],[393 m²],[Mehrfamilienhaus],n.a,[1990],[D],[Gas],[Zentralheizung]
3,277z85u,berlin,50,max. 10 km,1.950.000 €,[212.58 m²],[7 Zi.],[941 m²],[Einfamilienhaus],n.a,[1956],[C],n.a,"[offener Kamin, Zentralheizung]"
4,26zkl56,berlin,50,max. 10 km,429.000 €,[97 m²],[4 Zi.],[407 m²],[Einfamilienhaus],n.a,[1958],[E],[Öl],[Zentralheizung]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,27szw5w,stuttgart,50,max. 30 km,745.000 €,[103 m²],[4.5 Zi.],[314 m²],[Reihenendhaus],n.a,n.a,n.a,[Öl],n.a
6996,2722w5t,stuttgart,50,max. 30 km,680.000 €,[213 m²],[8 Zi.],[331 m²],[Doppelhaushälfte],n.a,[1904],[E],[Gas],[Zentralheizung]
6997,28z8g52,stuttgart,50,max. 30 km,378.000 €,[104 m²],[6 Zi.],[447 m²],[Mehrfamilienhaus],n.a,[1958],[F],[Öl],[Ofen]
6998,27fps5t,stuttgart,50,max. 30 km,659.000 €,[169 m²],[6 Zi.],[223 m²],n.a,n.a,[1995],n.a,[Gas],[Zentralheizung]
